In [1]:
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
import numpy as np
from numpy.linalg import norm
import os
from tqdm import tqdm
import pickle

In [2]:
model = ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))
model.trainable = False

model = tensorflow.keras.Sequential([
    model,
    GlobalMaxPooling2D()
])

#print(model.summary())

2023-11-28 00:03:10.373481: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2023-11-28 00:03:10.373513: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2023-11-28 00:03:10.373521: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2023-11-28 00:03:10.373568: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-11-28 00:03:10.373594: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
def extract_features(img_path,model):
    img = image.load_img(img_path,target_size=(224,224))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    result = model.predict(preprocessed_img).flatten()
    normalized_result = result / norm(result)

    return normalized_result

In [4]:
filenames = []

for file in os.listdir('downloaded_images'):
    filenames.append(os.path.join('downloaded_images',file))

feature_list = []

for file in filenames:
    try:
        feature_list.append(extract_features(file,model))
    except OSError as e:
        print(f"Error processing image file: {file}. {str(e)}")


1/1 [==============================] - 0s 28ms/step
Error processing image file: downloaded_images/30869.jpg. image file is truncated (18 bytes not processed)
1/1 [==============================] - 0s 29ms/step
Error processing image file: downloaded_images/80105.jpg. image file is truncated (17 bytes not processed)
1/1 [==============================] - 0s 36ms/step


In [5]:

pickle.dump(feature_list,open('embeddings_new.pkl','wb'))
pickle.dump(filenames,open('filenames_new.pkl','wb'))